## imports

In [1]:
import pandas as pd
import datetime
import tqdm

## choose variables


In [15]:
selected_country = 'Country A'

filter_vehiclegroup = False
selcted_vehiclegroup = 'vehiclegroup01' # do not touch!!
filename = 'current_run_'

### wheel
use_all_features = True # if false, assign checkup_features here
checkup_features = ['quota__vehiclegroup01__aq'],['sales_flow_monthly__vehiclegroup01__cp8']
data_input = 'all' #all, internal, external
predict_future_values = False #after row 258

#methods
mean_per_month = True
median_per_month = True
interpolation_linear = True
interpolation_spline = True
fill_zero = True

In [16]:
df = pd.read_csv("../Data/kitCapstoneJan24_data.csv", sep=";")

In [17]:
#Get rows with for country A

df_country_a = df[df['countryname'] == selected_country]
df_country_a

,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_actuals_monthly__vehiclegroup02__orderintake,sales_actuals_monthly__vehiclegroup02__retail,sales_actuals_monthly__vehiclegroup03__orderintake,sales_actuals_monthly__vehiclegroup03__retail,sales_actuals_monthly__vehiclegroup04__orderintake,sales_actuals_monthly__vehiclegroup04__retail,...,registrations__buildtype_suv,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi
0,1,Country A,296.0,352.0,739.0,471.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Country A,414.0,372.0,904.0,528.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Country A,405.0,604.0,1000.0,953.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Country A,359.0,483.0,828.0,972.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Country A,403.0,474.0,594.0,900.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,357,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,358,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,359,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# add additional feature month_normalized
df_country_a['month_normalized'] = df_country_a['month'] % 12

df_country_a = df_country_a[['month_normalized','month', 'countryname'] + [col for col in df_country_a.columns if col not in ['month_normalized','month', 'countryname']]]

In [19]:

if filter_vehiclegroup:
    ## select vehiclegroups
    # delete all columns that are for a diffe
    df1 = df_country_a

    # create new dataframe with month and all columns containing vehiclegroup01
    df2 = df1.filter(regex=selcted_vehiclegroup)
    df2.head()

    # drop columns in df1 which contain vehiclegroup
    df3 = df1.drop(df1.filter(regex='vehiclegroup').columns, axis=1)
    df3.head()

    # merge df2 and df3 by index
    df4 = df2.merge(df3, left_index=True, right_index=True)
    df4.head()

    # is there a column month in df4?
    df4.columns.str.contains('month').any()
    # column month should be the first column, second column is countryname
    df4 = df4[['month_normalized','month', 'countryname'] + [col for col in df4.columns if col not in ['month_normalized','month', 'countryname']]]

    # drop rows after row 258
    #test = df4.drop(df4.index[258:])
    test = df4
else:
    test = df_country_a 

In [20]:
df_overall = test

In [21]:
df_overall

,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_actuals_monthly__vehiclegroup02__orderintake,sales_actuals_monthly__vehiclegroup02__retail,sales_actuals_monthly__vehiclegroup03__orderintake,sales_actuals_monthly__vehiclegroup03__retail,sales_actuals_monthly__vehiclegroup04__orderintake,...,registrations__buildtype_suv,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi
0,1,1,Country A,296.0,352.0,739.0,471.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,Country A,414.0,372.0,904.0,528.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,3,Country A,405.0,604.0,1000.0,953.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,Country A,359.0,483.0,828.0,972.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,Country A,403.0,474.0,594.0,900.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,8,356,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,9,357,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,10,358,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,11,359,Country A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# dataframe with first 18 columns (internal data)
df_internal = test.iloc[:, :21]
df_internal.head()

,month_normalized,month,countryname,sales_actuals_monthly__vehiclegroup01__orderintake,sales_actuals_monthly__vehiclegroup01__retail,sales_actuals_monthly__vehiclegroup02__orderintake,sales_actuals_monthly__vehiclegroup02__retail,sales_actuals_monthly__vehiclegroup03__orderintake,sales_actuals_monthly__vehiclegroup03__retail,sales_actuals_monthly__vehiclegroup04__orderintake,...,sales_actuals_monthly__vehiclegroup05__orderintake,sales_actuals_monthly__vehiclegroup05__retail,sales_actuals_monthly__vehiclegroup06__orderintake,sales_actuals_monthly__vehiclegroup06__retail,sales_actuals_monthly__vehiclegroup07__orderintake,sales_actuals_monthly__vehiclegroup07__retail,sales_actuals_monthly__vehiclegroup08__orderintake,sales_actuals_monthly__vehiclegroup08__retail,sales_actuals_monthly__vehiclegroup09__orderintake,sales_actuals_monthly__vehiclegroup09__retail
0,1,1,Country A,296.0,352.0,739.0,471.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,607.0,408.0,614.0,495.0,NaN,NaN
1,2,2,Country A,414.0,372.0,904.0,528.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,731.0,474.0,768.0,534.0,NaN,NaN
2,3,3,Country A,405.0,604.0,1000.0,953.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1000.0,928.0,573.0,869.0,NaN,NaN
3,4,4,Country A,359.0,483.0,828.0,972.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,835.0,1000.0,464.0,820.0,NaN,NaN
4,5,5,Country A,403.0,474.0,594.0,900.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,494.0,873.0,487.0,827.0,NaN,NaN


In [23]:
# dataframe with external features
df_external = test.iloc[:, 21:]
df_external.loc[:, 'month_normalized'] = test['month_normalized']
df_external.head()

,sales_actuals_monthly__vehiclegroup10__orderintake,sales_actuals_monthly__vehiclegroup10__retail,sales_actuals_monthly__vehiclegroup11__orderintake,sales_actuals_monthly__vehiclegroup11__retail,sales_actuals_monthly__vehiclegroup12__orderintake,sales_actuals_monthly__vehiclegroup12__retail,sales_actuals_monthly__vehiclegroup13__orderintake,sales_actuals_monthly__vehiclegroup13__retail,sales_actuals_monthly__vehiclegroup14__orderintake,sales_actuals_monthly__vehiclegroup14__retail,...,registrations__buildtype_transport,registrations__buildtype_van,registrations__buildtype_cabrio,registrations__buildtype_stufenheck,registrations__buildtype_kurzheck,registrations__buildtype_other,registrations__buildtype_pickup,registrations__buildtype_coupe,registrations__buildtype_kombi,month_normalized
0,NaN,NaN,NaN,NaN,NaN,NaN,518.0,547.0,5.0,146.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,737.0,592.0,20.0,135.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,NaN,NaN,NaN,NaN,NaN,NaN,792.0,1000.0,12.0,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,NaN,NaN,NaN,NaN,NaN,NaN,706.0,858.0,7.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,NaN,NaN,NaN,NaN,NaN,NaN,783.0,836.0,19.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [24]:
country_a_y = df_country_a['sales_actuals_monthly__vehiclegroup01__orderintake']

### feature filling wheel

In [32]:
#import warnings
#warnings.filterwarnings("ignore") #turn of warnings

if data_input == 'all':
    input_df = df_overall
elif data_input == 'internal':
    input_df = df_internal
elif data_input == 'external':
    input_df = df_external

if not predict_future_values:
    input_df = input_df.head(258)  # Select the first 258 rows as the first part
    df_second_part = input_df.tail(102)  # Select the last 102 rows as the second part

if use_all_features:
    checkup_features = input_df.columns

# Create a progress bar
pbar = tqdm.tqdm(total=len(checkup_features))

for feature in checkup_features:
    if input_df[feature].isna().any():
        if mean_per_month:
            # add mean per month
            # Fill in missing values using the corresponding month values
            month_means = {}
            for i in range(len(input_df)):
                month = input_df.iloc[i]['month_normalized']
                month_values = input_df.loc[input_df['month_normalized'] == month, feature]
                if not month_values.isnull().values.all():
                    month_means[month] = month_values.mean()

            # Fill in missing values using the corresponding month values
            input_df[feature+'_meanbymonth']=input_df[feature].loc[:].fillna(input_df['month_normalized'].map(month_means))

        if median_per_month:
            # add median per month
            # Fill in missing values using the corresponding month values
            month_medians = {}
            for i in range(len(input_df)):
                month = input_df.iloc[i]['month_normalized']
                month_values = input_df.loc[input_df['month_normalized'] == month, feature]
                if not month_values.isnull().values.all():
                    month_medians[month] = month_values.median(skipna=True)
            
            input_df[feature + '_medianbymonth'] = (input_df[feature].loc[:].
                                                    fillna(input_df['month_normalized']
                                                           .map(month_medians)))

        if interpolation_linear:
            input_df[feature+'_interpolated_linear']=input_df[feature].interpolate(method='linear', limit_direction = 'both')

        if interpolation_spline:
            input_df[feature + '_interpolation_spline'] = input_df[feature].interpolate(method='spline', limit_direction = 'both', order=3)
    
        if fill_zero:
            input_df[feature+'_fill_zero']=input_df[feature].fillna(0)

        input_df.drop(columns=feature, inplace=True)
    pbar.update(1)

if not predict_future_values:       
    input_df = pd.concat([input_df, df_second_part], axis=0)  # Concatenate the two DataFrames along the row axis


output_df = input_df

# Close the progress bar
pbar.close()

est
     month_normalized  month countryname  \
0                   1      1   Country A   
1                   2      2   Country A   
2                   3      3   Country A   
3                   4      4   Country A   
4                   5      5   Country A   
..                ...    ...         ...   
253                 2    254   Country A   
254                 3    255   Country A   
255                 4    256   Country A   
256                 5    257   Country A   
257                 6    258   Country A   

     sales_actuals_monthly__vehiclegroup01__orderintake  \
0                                                296.0    
1                                                414.0    
2                                                405.0    
3                                                359.0    
4                                                403.0    
..                                                 ...    
253                                              531.0

  1%|          | 20/2370 [00:09<17:40,  2.22it/s]

  1%|          | 26/2370 [00:07<18:04,  2.16it/s]

In [29]:
output_df

NameError: name 'output_df' is not defined

In [ ]:
output_df.to_csv('../Data/'+filename+datetime.datetime.now().strftime("%H%M")+'_NaN_fillup.csv')

## NaN lists

hardcoded at the moment

In [ ]:
df_overall_future = df_overall.iloc[:259,]


# Assuming df_external_info is a DataFrame
nan_columns = df_overall_future.isna().any()
nan_counts = df_overall_future.isna().sum()


# Create a list of rows
rows = []
for col, count in nan_counts.items():
    if count > 50:
        row = {"name": col, "NaN Count": count}
        rows.append(row)

# Create a DataFrame
nan_info_df = pd.DataFrame(rows, columns=["name", "NaN Count"])

nan_info_df

nan_info_df.to_csv('../Data/'+filename+datetime.datetime.now().strftime("%H%M")+'nan_list.csv')

# Create a list of rows
rows = []
for col, count in nan_counts.items():
    if count <= 50:
        row = {"name": col, "NaN Count": count}
        rows.append(row)

# Create a DataFrame
not_nan_info_df = pd.DataFrame(rows, columns=["name", "NaN Count"])

not_nan_info_df

not_nan_info_df.to_csv('../Data/'+filename+datetime.datetime.now().strftime("%H%M")+'not_nan_list.csv')